In [1]:
from google.colab import files
uploaded = files.upload()

Saving gut.txt to gut.txt


In [28]:
text = open('gut.txt', 'r', encoding = 'utf-8').read()[2:]
len(text)

3227577

In [29]:
import string
import re

def preprocess_text(text):
    new_text = text.lower()
    #new_text = re.sub(r'[!?"<>:;,—–«»”“’@]','',new_text)
    new_text = new_text.translate(str.maketrans('', '', ',—*)[-];:/@!“?’#%=”‘('))
    new_text = re.sub(r"\s+",' ',new_text)
    del text
    return new_text

text = preprocess_text(text)
len(text)
#assert len(text) == 3141169

3130291

In [0]:
text = text.split('.')
text = [x.strip() for x in text[:1000]]

In [0]:
def flatten(list):
  for i in list:
    for j in i:
      yield j

In [0]:
from collections import Counter
import nltk
from sklearn.base import TransformerMixin


class BPE(TransformerMixin):
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}
        
    def fit(self, text):
        """
        fit itos and stoi
        text: list of strings 
        """
        
        text = [list(sentence) for sentence in text]
        self.itos = list(set(list(flatten(text))))
        for i, item in enumerate(self.itos):
          self.stoi[item] = i
        
        text = [[self.stoi[sym] for sym in sent] for sent in text]
        
        while len(self.itos) < self.vocab_size:
            freq = Counter(zip(list(flatten(text)), list(flatten(text))[1:]))
            new_token = freq.most_common(1)[0][0]
            new_id = len(self.itos)
            
            self.itos.append(new_token)
            self.stoi[new_token] = new_id
            
            
            for sent in text:
              indices = [i for i, x in enumerate(sent) if x == new_token[0]]
              removeid = []

              for index in indices:
                if index!=len(sent)-1:
                  if sent[index+1] == new_token[1]:
                    removeid.append(index+1)
                    sent[index] = new_id
              
              sent = [i for j, i in enumerate(sent) if j not in removeid]
            
        return self
    
    def transform(self, text):
        """
        convert text to a sequence of token ids
        text: list of strings
        """
        text = [list(sentence) for sentence in text]
        text = [[self.stoi[sym] for sym in sent] for sent in text]
        
        for sent in text:
          for el in range(len(self.itos)):
            indices = [i for i, x in enumerate(sent) if x == el]
  
            needed_itos = [k for k in self.itos if type(k)!=str and k[0] == el]
            for ix in indices:
              if ix<len(sent)-1:
                for item in needed_itos:
                  if sent[ix+1] == item[1]:
                    sent = sent[:ix]+[self.stoi[item]]+sent[ix+2:] 
                  
        return text
      
    def decode_token(self, tok):
        """
        tok: int or tuple
        """
        # and credits go to StackOverflow
        def recursive_dec(token): #token int or tuple
          if type(token) == int:
            sign = self.itos[token]
            if type(sign) == str:
              return sign 
            else:
              return recursive_dec(sign)
          elif type(token) == tuple:
            return recursive_dec(token[0]) + recursive_token_lookup(token[1]) 
        return recursive_dec(tok)
        result = []# TODO
        return recursive_dec(tok)
            
    def decode(self, text):
        """
        convert token ids into text
        """
        return ''.join(map(self.decode_token, text))
        
  
  
    
vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text)

In [0]:
assert bpe.decode(tokenized_text[0]) == text[0]

In [0]:
def smooth(count_current_3gram, count_all_3grams, tau, delta):
           return ((count_current_3gram + delta) ** (1/tau))/((count_all_3grams + delta) ** (1/tau))

In [0]:
import numpy as np
from itertools import tee, islice


start_token = vocab_size
end_token = vocab_size + 1

bpe.itos.append('@')
bpe.itos.append(' ')

bpe.stoi['@'] = start_token
bpe.stoi['.'] = end_token

def ngrams(lst, n):
  tlst = lst
  while True:
    a, b = tee(tlst)
    l = tuple(islice(a, n))
    if len(l) == n:
      yield l
      next(b)
      tlst = b
    else:
      break
        
    
class LM:
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.proba = []
        
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        
        text = [[start_token]+sent+[end_token] for sent in text]
        text = list(flatten(text))
        self.proba = Counter(ngrams(text, 3))
        return self
  
    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        
        #вероятност триграммы абс
        """
        counter_result = self.proba
        count_current_3gram = counter_result[(a,  b, c)]
        count_all_3grams = 0
        for count in counter_result:
          if count[0]==a and count[1]==b:
            count_all_3grams += counter_result[count]
        result = smooth(count_current_3gram, count_all_3grams, tau, self.delta)
        return result
    
    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        vector = []
        for item in range(self.vocab_size):
          vector.append(self.get_proba(a, b, item, tau=1))
        result = vector
        
        return result
      
    
    
lm = LM(vocab_size, 1).fit(tokenized_text)

In [0]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """

    beam = [(input_seq, 1)]
    
    
    for i in range(max_len-len(input_seq)):
      candidates = []
      candidates_proba = [] #LM(vocab_size, 1).infer(a, b, tau=1)
    
      for item in beam:
          snt = item[0]
          snt_proba = item[1]
          x = snt[-2]
          y = snt[-1]
          if y == end_token:
            continue
          else:    
              proba = lm.infer(x, y, tau) 
              sorted_probs = sorted([(e,i) for i,e in enumerate(proba)], reverse = True)
              best_k = [bpe.itos[ix[1]] for ix in sorted_probs[:k]]
              
              for token in best_k:
                candidates.append(item[0]+[bpe.stoi[token]])
                candidates_proba.append(sorted_probs[bpe.stoi[token]][0])
                
      beam = [(candidates[i],candidates_proba[i]) for i in range(k)]
      
    return beam
    

In [37]:

input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)

for item in result:
  print(bpe.decode(item[0]), item[1])
    

horse and  0.0010787486515641855
horse andr 0.0010787486515641855
horse ande 0.0010787486515641855
horse ands 0.0010787486515641855
horse ando 0.0010787486515641855
horse andi 0.0010787486515641855
horse and  0.0010787486515641855
horse andl 0.0010787486515641855
horse andu 0.0010787486515641855
horse andf 0.0010787486515641855


In [38]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
print(result)

for item in result:
  print(bpe.decode(item[0]), item[1])
    

[([18, 37, 23, 38, 21, 34, 34, 38, 6, 18], 0.001122334455667789), ([18, 37, 23, 38, 21, 34, 34, 38, 6, 30], 0.001122334455667789), ([18, 37, 23, 38, 21, 34, 34, 38, 6, 40], 0.001122334455667789), ([18, 37, 23, 38, 21, 34, 34, 38, 6, 22], 0.001122334455667789), ([18, 37, 23, 38, 21, 34, 34, 38, 6, 23], 0.001122334455667789), ([18, 37, 23, 38, 21, 34, 34, 38, 6, 39], 0.001122334455667789), ([18, 37, 23, 38, 21, 34, 34, 38, 6, 37], 0.001122334455667789), ([18, 37, 23, 38, 21, 34, 34, 38, 6, 34], 0.001122334455667789), ([18, 37, 23, 38, 21, 34, 34, 38, 6, 2], 0.10886644219977554), ([18, 37, 23, 38, 21, 34, 34, 38, 6, 38], 0.001122334455667789)]
her xii ch 0.001122334455667789
her xii co 0.001122334455667789
her xii ca 0.001122334455667789
her xii cl 0.001122334455667789
her xii cr 0.001122334455667789
her xii cu 0.001122334455667789
her xii ce 0.001122334455667789
her xii ci 0.001122334455667789
her xii cy 0.10886644219977554
her xii c  0.001122334455667789


In [39]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)

for item in result:
  print(bpe.decode(item[0]), item[1])
    

what the a 0.0003214400514304082
what the t 0.0022500803600128574
what the w 0.0003214400514304082
what the s 0.008678881388621022
what the h 0.021215043394406944
what the o 0.0003214400514304082
what the i 0.0003214400514304082
what the p 0.0003214400514304082
what the c 0.04885888781742205
what the b 0.0003214400514304082


In [40]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
for item in result:
  print(bpe.decode(item[0]), item[1])
    

gun the an 0.000998003992015968
gun the a  0.000499001996007984
gun the al 0.000499001996007984
gun the as 0.000499001996007984
gun the at 0.000499001996007984
gun the ar 0.000499001996007984
gun the ab 0.000499001996007984
gun the am 0.014970059880239521
gun the ag 0.00998003992015968
gun the ad 0.000499001996007984


In [41]:
from math import log

def perplexity(snt, lm):
    """
    snt: sequence of token ids
    lm: language model
    """
    text = list(snt)
    counter_result = Counter(ngrams(text, 3))
    
    trigrams_probs = [log(lm.get_proba(k[0],k[1],k[2], tau=1)) for k in counter_result.keys()]
    m = len(snt)
    result = 2 **(-1/m*(sum(trigrams_probs)))
    return result

perplexity(tokenized_text[0], lm)

3.248088081771401